In [1]:
import torch

# 检查CUDA是否可用
if torch.cuda.is_available():
    # 获取可用的GPU设备数量
    gpu_count = torch.cuda.device_count()
    print(f"发现 {gpu_count} 个可用的GPU 设备.")

    # 遍历并打印每个GPU设备的名称
    for i in range(gpu_count):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i + 1}: {gpu_name}")
else:
    print("未发现可用的GPU设备.")


发现 4 个可用的GPU 设备.
GPU 1: NVIDIA A800-SXM4-80GB
GPU 2: NVIDIA A800-SXM4-80GB
GPU 3: NVIDIA A800-SXM4-80GB
GPU 4: NVIDIA A800-SXM4-80GB


In [2]:
import copy
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge
# from bert_score import score
import statistics
from ast import literal_eval
import functools
import json
import os
import random
import wget
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F

random.seed(8888)
torch.manual_seed(8888)
random.seed(8888)
np.random.seed(8888)

if torch.cuda.is_available():
    torch.cuda.manual_seed(8888)
    torch.cuda.manual_seed_all(8888)


from tqdm import tqdm

torch.set_grad_enabled(False)
tqdm.pandas()

In [3]:
torch.cuda.set_device(2)


from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from os.path import join

# Get CounterFact data for GPT2-xl, from the ROME repository.
#wget.download("https://rome.baulab.info/data/dsets/known_1000.json")


model_dir = "/U_PZL2023ZZ0005/yhhong/transformers"
model_name = "Qwen1.5-7B-Chat" #"chatglm3-6b" #"Llama-2-7b-chat-hf" #"Qwen1.5-7B-Chat" 


model = AutoModelForCausalLM.from_pretrained(
    join(model_dir, model_name),
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
);

tokenizer = AutoTokenizer.from_pretrained(join(model_dir, model_name))
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

model.to('cuda');


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
model.config

Qwen2Config {
  "_name_or_path": "/U_PZL2023ZZ0005/yhhong/transformers/Qwen1.5-7B-Chat",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 32768,
  "max_window_layers": 28,
  "model_type": "qwen2",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "rms_norm_eps": 1e-06,
  "rope_theta": 1000000.0,
  "sliding_window": 32768,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

In [5]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-31): 32 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): L

In [23]:
INPUT = ["汉堡","薯条"]

In [24]:
encoded_id1 = tokenizer.encode(INPUT[0])[0]
encoded_id2 = tokenizer.encode(INPUT[1])[0]
print(encoded_id1)
print(encoded_id2)

117335
103532


In [25]:
E = model.get_output_embeddings().weight.detach() 
#E = model.transformer.output_layer.weight.detach()


model_state_dict = model.state_dict()
#print(model_state_dict)
values_list1 = []
values_list2 = []
ix = 0
top_k = 100
list_common_nums = []

for key, values in model_state_dict.items():
    if 'mlp.down_proj' in key:  #llama
    #if 'dense_4h_to_h' in key:  #chatglm
        print(f"Key: {key}")
        values_map = values.T.matmul(E.T)
        #print(values.T.matmul(E.T).shape)  #理解为维度有11008个

        col1 = values_map[:, encoded_id1]  #27675:_Hog  10686:_Harry
        sorted_values, sorted_indices = torch.sort(col1, descending=True)
        topk_indices1 = sorted_indices[:top_k]
        values_list1.append(topk_indices1)


        col2 = values_map[:, encoded_id2]  
        sorted_values, sorted_indices = torch.sort(col2, descending=True)
        topk_indices2 = sorted_indices[:top_k]
        values_list2.append(topk_indices2)

        common_nums = set(topk_indices1.tolist()) & set(topk_indices2.tolist())
        print(ix,'层中',f"两个列表中共同的数字为: {common_nums}")
        list_common_nums.append(list(common_nums))

        if len(list_common_nums[ix]) != 0:
            for item in list_common_nums[ix]:
                _, sorted_indices_item = torch.sort(values_map[item,:], descending=True) 
                sorted_indices_item = sorted_indices_item.cpu().numpy()
                #print(f'in Layer{ix} ,', f'sorted_indices_dimension{item}: ',tokenizer.convert_ids_to_tokens(sorted_indices_ite    m[:200]))
                print(f'in Layer{ix} ,', f'sorted_indices_dimension{item}: ',[tokenizer.decode(t) for t in sorted_indices_item[:200]]) #qwen
        ix +=1    


#convert_ids_to_tokens

Key: model.layers.0.mlp.down_proj.weight
0 层中 两个列表中共同的数字为: {7046}
in Layer0 , sorted_indices_dimension7046:  ['udoku', " ''){\n", 'imitive', '.getSeconds', 'nyder', 'infeld', '.myapplication', 'IALOG', '农副', 'oenix', '<TEntity', '/API', "'],['", 'prix', '&action', '_DGRAM', ' disponíveis', 'apur', '污染物', '.nih', ' OVERRIDE', '黢', ' 😉\n\n', ' onFocus', 'ABEL', '马克', '}->', ' dhe', 'lien', ' Phrase', '\\admin', " '');", '.club', '("[%', 'versible', '?url', '售', '佔', '.defineProperty', ' }},\n', '_EXTENSION', 'reich', ' LoginActivity', '螫', 'のも', '东路', '嵛', ' marc', 'egend', '农场', '绀', '辿', '_iterations', '}->{', ' gettext', 'UIViewController', ':path', '案', '促成', 'ifecycle', 'WINDOWS', ');">\n', '电子信息', '恨不得', 'OutputStream', ' DISCLAIM', '薹', 'ifetime', '.Caption', '")==', 'INavigation', '_REUSE', ' ******************************************************************************/\n\n', '平均每', '妯', 'そうで', 'phins', '])):\n', '=""/>\n', 'keterangan', ' eBooks', ' Honour', '向往', '所在的', '彀', '

In [ ]:
Generation Testing

In [16]:
question = "What is Mcdonald‘s?"
inputs = tokenizer(f"Question: {question}\n Answer:", return_tensors="pt")
input_ids = inputs["input_ids"].to('cuda')

with torch.no_grad():
    generation_output = model.generate(  #mt.model
        input_ids=input_ids,
        return_dict_in_generate=True,
        do_sample = False,
        max_new_tokens=100,
    )
s = generation_output.sequences[0]
output = tokenizer.decode(s)
output

"Question: What is Mcdonald‘s?\n Answer: McDonald's is a fast-food restaurant chain that serves a variety of menu items, including burgers, fries, salads, and milkshakes. It was founded in 1955 by Ray Kroc in the United States and has since become one of the world's largest and most recognizable brands. McDonald's offers customizable meals, breakfast options, and kid-friendly dining experiences. The restaurants are known for their distinctive golden arches logo and often have drive-through service, as well as indoor seating. In"